In [1]:
!pip install pandas_gbq
!pip install mysql-connector-python

In [15]:
import pandas as pd
from pandas_gbq import to_gbq
from google.cloud import bigquery
import pandas as pd

# MySQL
import mysql
import pandas as pd
import mysql.connector as connection

#BigQuery
credentials = 'data-analyst-prj-bigquery-a71087fac92a.json'
bqclient = bigquery.Client.from_service_account_json(credentials)

#Crear el Schema destino
project_id = 'data-analyst-prj'
dataset_id = 'python_mysql_ecommerce_raw'


# MySQL Connection
db_mysql = mysql.connector.connect(
    host="72.167.124.128",
    user="ecommerce_prd_admin",
    password="L0GM4CAdcMVNrMBFZKzYVGEZlbxQ",
    database="ecommerce_prd"
)


def fix_date_cols(df, tz='UTC'):
    cols = df.select_dtypes(include=['datetime64[ns]']).columns
    for col in cols:
        df[col] = df[col].dt.tz_localize(tz)
    return df


In [16]:
import warnings
warnings.filterwarnings("ignore")


# MySQL
query = '''SELECT
            table_schema
              , table_name
              , table_rows
            FROM information_schema.tables
            WHERE TABLE_TYPE = "BASE TABLE"
        '''
df = pd.read_sql(query, db_mysql)

In [17]:
df

,table_schema,table_name,table_rows
0,ecommerce_prd,customers,98838
1,ecommerce_prd,geolocation,983484
2,ecommerce_prd,order_items,111588
3,ecommerce_prd,order_payments,103454
4,ecommerce_prd,order_reviews,98443
5,ecommerce_prd,orders,98789
6,ecommerce_prd,product_category_name_translation,71
7,ecommerce_prd,products,32710
8,ecommerce_prd,sellers,3095


In [18]:
for index, row in df.T.items():
    #read from mysql
    schema = row['table_schema']
    table = row['table_name']
    query = "SELECT * FROM {0}".format(table)
    print("loading...{0}".format(table))
    df_mysql = pd.read_sql(query, db_mysql)
    df_mysql = fix_date_cols(df_mysql)
    #write to bigquery
    df_mysql = df_mysql.reset_index(drop=True)
    df_mysql.columns = map(lambda x: str(x).upper(), df_mysql.columns)
    print("writing on BigQuery..".format(table))

    table_id = project_id + '.' + dataset_id + '.' + table
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    job = bqclient.load_table_from_dataframe(df_mysql, table_id, job_config=job_config)  

    # Wait for the job to complete and then show the job results
    job.result()

    # Read back the properties of the table
    table = bqclient.get_table(table_id)
    print("Table:", table.table_id, "has", table.num_rows, "rows and", len(table.schema), "columns")

    print("-----------------------------")
        
db_mysql.close()

loading...customers
writing on BigQuery..
Table: customers has 99441 rows and 5 columns
-----------------------------
loading...geolocation
writing on BigQuery..
Table: geolocation has 1000163 rows and 7 columns
-----------------------------
loading...order_items
writing on BigQuery..
Table: order_items has 112650 rows and 7 columns
-----------------------------
loading...order_payments
writing on BigQuery..
Table: order_payments has 103886 rows and 5 columns
-----------------------------
loading...order_reviews
writing on BigQuery..
Table: order_reviews has 99224 rows and 7 columns
-----------------------------
loading...orders
writing on BigQuery..
Table: orders has 99441 rows and 8 columns
-----------------------------
loading...product_category_name_translation
writing on BigQuery..
Table: product_category_name_translation has 71 rows and 2 columns
-----------------------------
loading...products
writing on BigQuery..
Table: products has 32951 rows and 9 columns
-------------------